In [ ]:
from data_parser import DataParser
from datetime import datetime
import pandas as pd
import numpy as np
import copy

In [ ]:
dp = DataParser(filename='../data/GBPUSD.csv')

In [ ]:
start_datetime = datetime.strptime('20021001 223600','%Y%m%d %H%M%S')
end_datetime = datetime.strptime('20120228 183600','%Y%m%d %H%M%S')

In [ ]:
dp.filter_by_date(start_datetime, end_datetime)

In [ ]:
dp.sample_by_time('40min', 2160000000000)

In [ ]:

Xn,yn = dp.get_close_price_numpy_array(1, 22, 21, 30,36)

In [ ]:
Xn1 = dp.tranform_X_to_returns(Xn)

In [ ]:
Xn2 = dp.transform_X_with_IWMA(Xn1)

In [ ]:
X_norm, y_norm = dp.normalize_data(Xn2, yn)

In [ ]:
print(X_norm.shape)
print(y_norm.shape)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)
train_X_in, test_X_in, train_y_in, test_y_in = train_test_split(train_X, train_y, test_size=0.1, random_state=42)

In [ ]:
from ann import ANN

In [ ]:
from GA import GeneticAlgorithm

In [ ]:
def fitness(chromosome):
    #chromosomes have 8 bits. first 4 bits are the hidden layer 1, next 4 bits are hidden layer 2. but chromosome is a numpy array
    hidden_layer_1 = 0
    hidden_layer_2 = 0
    for i in range(4):
        hidden_layer_1 += chromosome[i] * (2 ** (3-i))
        hidden_layer_2 += chromosome[i+4] * (2 ** (3-i))
#take mod with 20
    hidden_layer_1 = (hidden_layer_1 % 20) + 1
    hidden_layer_2 = (hidden_layer_2 % 20) + 1
    ann = ANN(hidden_layer_1, hidden_layer_2, 20, 1, 'tanh','lbfgs', 1000,42)
    ann.fit(train_X_in, train_y_in)
    y_pred = ann.predict(test_X_in)
    nc = ann.num_correct(test_y_in, y_pred)
    return nc

In [ ]:
ga = GeneticAlgorithm(population_size=30, chromosome_length=8, crossover_rate=0.8, mutation_rate=0.2, num_generations=40, stall_generations=15)
ga.fitness_function = fitness
best_chromosome, best_fitness = ga.run()

In [ ]:
print(best_chromosome)

In [ ]:
hidden_layer_1 = 0
hidden_layer_2 = 0
for i in range(4):
    hidden_layer_1 += best_chromosome[i] * (2 ** (3-i))
    hidden_layer_2 += best_chromosome[i+4] * (2 ** (3-i))
#take mod with 20
hidden_layer_1 = (hidden_layer_1 % 20) + 1
hidden_layer_2 = (hidden_layer_2 % 20) + 1
ann = ANN(hidden_layer_1, hidden_layer_2, 20, 1, 'tanh', 'adam', 100, 42)
ann.fit(train_X_in, train_y_in)
y_pred = ann.predict(test_X)
nc = ann.num_correct(test_y, y_pred)
print(nc)

In [ ]:
print(ann.num_wrong(test_y, y_pred))